In [ ]:
#  # Google Colab doesn't come with this package installed, so we'll install it.
# !pip install keras-tuner

In [ ]:
import keras_tuner as kt

In [ ]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [ ]:
#  Import and read the charity_data.csv

df = pd.read_csv("Resources/charity_data.csv")
print(df.info())
df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

df = df[["AFFILIATION", "ORGANIZATION", "APPLICATION_TYPE", "IS_SUCCESSFUL"]]
df

In [ ]:
# Determine the number of unique values in each column.

for col in df.columns:
    print(f"{col} \n{df[col].value_counts()}\n\n")

In [ ]:
# Look at APPLICATION_TYPE value counts for binning

df["APPLICATION_TYPE"].value_counts()

In [ ]:
application_types_to_replace = (df["APPLICATION_TYPE"].value_counts().loc[lambda x: x < 500]).keys().tolist()

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace

In [ ]:
# Replace in dataframe

for code in application_types_to_replace:
    print(code)
    df["APPLICATION_TYPE"] = df["APPLICATION_TYPE"].replace(code, "Other")
        
# Check to make sure binning was successful

df["APPLICATION_TYPE"].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning

df["CLASSIFICATION"].value_counts()

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1

classifications_to_replace = (df["CLASSIFICATION"].value_counts().loc[lambda x: x < 1000]).keys().tolist()

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Replace in dataframe

for code in classifications_to_replace:
    df["CLASSIFICATION"] = df["CLASSIFICATION"].replace(code, "Other")
    
# Check to make sure binning was successful

df["CLASSIFICATION"].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`

alphasoup_cat = df.dtypes[df.dtypes == "object"].index.tolist() 
alphasoup_cat

In [ ]:
df[alphasoup_cat].nunique() 

In [ ]:
df1 = pd.get_dummies(df["APPLICATION_TYPE"], prefix="APP_TYPE", prefix_sep='_')
df2 = pd.get_dummies(df["AFFILIATION"], prefix="AFFIL", prefix_sep='_')
df3 = pd.get_dummies(df["ORGANIZATION"], prefix="ORG", prefix_sep='_')

In [ ]:
enc_df = pd.concat([df1, df2, df3, df["IS_SUCCESSFUL"]], axis=1)
enc_df.info()

In [ ]:
# Split our preprocessed data into our features and target arrays

y = enc_df["IS_SUCCESSFUL"].values
X = enc_df.drop(["IS_SUCCESSFUL"], axis=1).values

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0) 

In [ ]:
# Create a StandardScaler instances

scaler = StandardScaler()

# Fit the StandardScaler

X_scaler = scaler.fit(X_train)

# Scale the data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

print(X_train_scaled.shape)
print(X_test_scaled.shape)

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options

def create_model(hp):
    
    # Instantiate a Sequential model
    
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers.
    
    activation = hp.Choice('activation', ['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide the number of neurons in first layer and also
    # the activation function. 
    
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=27))

    # Allow kerastuner to decide the number of hidden layers and number of 
    # neurons in each one
    
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    # Define the output layer
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [ ]:
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [ ]:
# Run the kerastuner search for best hyperparameters

tuner.search(X_train_scaled, y_train,epochs=20, validation_data=(X_test_scaled, y_test))

In [ ]:
 # Get best model hyperparameters
 
best_hyper = tuner.get_best_hyperparameters(1)[0]

best_hyper.values

In [ ]:
 # Evaluate best model against full test data
 
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
 # Summarize the best model
 
best_model.summary()

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=5, input_dim=27, activation="tanh"))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=7, activation="tanh"))

# Third hidden layer

nn.add(tf.keras.layers.Dense(units=3, activation="tanh"))

# Fourth hidden layer

nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model

nn.summary()

In [ ]:
# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [ ]:
# Train the model

fit_model = nn.fit(X_train_scaled, y_train, epochs=10) 

In [ ]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file

nn.save("nn_optimized.h5")